<a href="https://colab.research.google.com/github/bhaveshgupta01/BCancerGogo/blob/main/GOGO_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input


# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 16

def label_images(directory, target_size=(img_width, img_height)):
    images = []
    labels = []
    class_labels = {'malignant': 0, 'benign': 1, 'normal': 2}

    for class_label, class_index in class_labels.items():
        class_path = os.path.join(directory, class_label)
        for filename in os.listdir(class_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(class_path, filename)
                image = cv2.imread(file_path)  # You can use PIL.Image.open() as well
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
                image = cv2.resize(image, target_size)  # Resize the image to target size
                image = preprocess_input(image)  # Preprocess the image for VGG16
                images.append(image)
                labels.append(class_index)

    return np.array(images), np.array(labels)


# Example usage:
directory_path = '/content/drive/MyDrive/MIASdata'
X, y = label_images(directory_path, target_size=(img_width, img_height))

# X contains the resized images, and y contains the corresponding labels
print(f"Total Images: {len(X)}")
print(f"Shape of an Image: {X[0].shape}")
print(f"Labels: {y}")


from sklearn.model_selection import train_test_split

# Assuming X and y are the images and labels obtained from the previous code
# X, y = label_images(directory_path)

# Split the data into 90% training and 10% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Print the sizes of the training and testing sets
print(f"Training Set: {len(X_train)} samples")
print(f"Testing Set: {len(X_test)} samples")


Total Images: 321
Shape of an Image: (224, 224, 3)
Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Training Set: 288 samples
Testing Set: 33 samples


In [ ]:
from tensorflow.keras.utils import to_categorical

def build_resnet_model(learn_rate=1e-4, momentum=0.9):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    # Freeze all layers except the last three
    for layer in base_model.layers[:-3]:
        layer.trainable = False

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(layers.Dense(num_classes, activation='softmax'))  # Change activation to 'softmax'

    optimizer = SGD(learning_rate=learn_rate, momentum=momentum)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# ... (your data loading and preprocessing code)
resnet_model = build_resnet_model()
# Number of folds for k-fold cross-validation
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(kfold.split(X, y), 1):
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Convert labels to one-hot encoding
    y_train_fold_one_hot = to_categorical(y_train_fold, num_classes=num_classes)
    y_val_fold_one_hot = to_categorical(y_val_fold, num_classes=num_classes)

    # Build a new ResNet50 model for each fold
    # resnet_model = build_resnet_model()

    # Train the ResNet50 model
    class_weights = {0: 4.0, 1: 4.0, 2: 1.0}  # Adjust the weights based on class imbalance

    history = resnet_model.fit(
        X_train_fold,
        y_train_fold_one_hot,
        epochs=10,
        validation_data=(X_val_fold, y_val_fold_one_hot),
        batch_size=32,
        verbose=1,
        class_weight=class_weights
    )

    # Evaluate the model on the validation set
    val_loss, val_acc = resnet_model.evaluate(X_val_fold, y_val_fold_one_hot, verbose=0)
    print(f"Validation Accuracy for Fold {fold}: {val_acc * 100:.2f}%")


Epoch 1/10
8/8 [==============================] - 64s 8s/step - loss: 3.0748 - accuracy: 0.3320 - val_loss: 1.7880 - val_accuracy: 0.1846
Epoch 2/10
8/8 [==============================] - 68s 9s/step - loss: 2.5082 - accuracy: 0.4023 - val_loss: 1.6879 - val_accuracy: 0.1846
Epoch 3/10
8/8 [==============================] - 67s 9s/step - loss: 2.1145 - accuracy: 0.4727 - val_loss: 1.6134 - val_accuracy: 0.1846
Epoch 4/10
8/8 [==============================] - 68s 9s/step - loss: 1.7305 - accuracy: 0.5469 - val_loss: 1.5247 - val_accuracy: 0.1846
Epoch 5/10
8/8 [==============================] - 71s 9s/step - loss: 1.5634 - accuracy: 0.5977 - val_loss: 1.4439 - val_accuracy: 0.1846
Epoch 6/10
8/8 [==============================] - 67s 9s/step - loss: 1.3752 - accuracy: 0.6367 - val_loss: 1.3893 - val_accuracy: 0.2154
Epoch 7/10
8/8 [==============================] - 68s 9s/step - loss: 1.2751 - accuracy: 0.6406 - val_loss: 1.3489 - val_accuracy: 0.2308
Epoch 8/10
8/8 [==================

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_test_one_hot=to_categorical(y_test, num_classes=num_classes)

# Function to calculate and print evaluation metrics
def evaluate_model_multi_class(model, X, y_true):
    # Predictions
    y_pred = model.predict(X)

    # Convert one-hot encoding to class labels
    y_true_labels = np.argmax(y_true, axis=1)
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true_labels, y_pred_labels)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # Confusion Matrix
    cm = confusion_matrix(y_true_labels, y_pred_labels)
    print("Confusion Matrix:")
    print(cm)

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_true_labels, y_pred_labels))

# Assuming you have trained the model 'inception_model' and loaded the test set 'X_val_fold', 'y_val_fold_one_hot'
evaluate_model_multi_class(resnet_model, X_test, y_test_one_hot)


2/2 [==============================] - 7s 167ms/step
Accuracy: 96.97%
Confusion Matrix:
[[ 4  0  0]
 [ 0  9  0]
 [ 0  1 19]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       0.90      1.00      0.95         9
           2       1.00      0.95      0.97        20

    accuracy                           0.97        33
   macro avg       0.97      0.98      0.97        33
weighted avg       0.97      0.97      0.97        33



In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Function to calculate and print evaluation metrics
def evaluate_model_multi_class(model, X, y_true):
    # Predictions
    y_pred = model.predict(X)

    # Convert one-hot encoding to class labels
    y_true_labels = np.argmax(y_true, axis=1)
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true_labels, y_pred_labels)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # Confusion Matrix
    cm = confusion_matrix(y_true_labels, y_pred_labels)
    print("Confusion Matrix:")
    print(cm)

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_true_labels, y_pred_labels))

# Assuming you have trained the model 'inception_model' and loaded the test set 'X_val_fold', 'y_val_fold_one_hot'
evaluate_model_multi_class(resnet_model, X_val_fold, y_val_fold_one_hot)


2/2 [==============================] - 13s 6s/step
Accuracy: 85.94%
Confusion Matrix:
[[10  0  0]
 [ 0 12  0]
 [ 5  4 33]]
Classification Report:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80        10
           1       0.75      1.00      0.86        12
           2       1.00      0.79      0.88        42

    accuracy                           0.86        64
   macro avg       0.81      0.93      0.85        64
weighted avg       0.90      0.86      0.86        64



`\/` IGNORE

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical

# Assuming you have three classes (malignant, benign, normal)
num_classes = 3

# Number of folds for k-fold cross-validation
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(kfold.split(X_train, y_train), 1):
    print(f"\nFold {fold} - Training Set: {len(X_train_fold)} samples, Validation Set: {len(X_val_fold)} samples")

    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Build a new ResNet50 model for each fold
    resnet_model = build_resnet_model()

    # Convert labels to one-hot encoding
    y_train_fold_one_hot = to_categorical(y_train_fold, num_classes=num_classes)
    y_val_fold_one_hot = to_categorical(y_val_fold, num_classes=num_classes)

    # Train the ResNet50 model
    class_weights = {0: 4.0, 1: 4.0, 2: 1.0}  # Adjust the weights based on class imbalance

    history = resnet_model.fit(
        X_train_fold,
        y_train_fold_one_hot,
        epochs=10,  # Adjust the number of epochs as needed
        validation_data=(X_val_fold, y_val_fold),
        batch_size=32,  # Adjust the batch size as needed
        verbose=1,
        class_weight=class_weights
    )

    # Evaluate the model on the validation set
    val_loss, val_acc = resnet_model.evaluate(X_val_fold, y_val_fold, verbose=0)
    print(f"Validation Accuracy for Fold {fold + 1}: {val_acc * 100:.2f}%")



Fold 1 - Training Set: 230 samples, Validation Set: 58 samples
Epoch 1/10
8/8 [==============================] - ETA: 0s - loss: 2.8963 - accuracy: 0.3870

ValueError: ignored

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

# Assuming you have three classes (malignant, benign, normal)
num_classes = 3

# Define the base model (ResNet50 without top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

def build_resnet_model(learn_rate=1e-4, momentum=0.9):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    # Freeze all layers except the last three
    for layer in base_model.layers[:-3]:
        layer.trainable = False

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(layers.Dense(num_classes, activation='softmax'))  # Change activation to 'softmax'

    optimizer = SGD(learning_rate=learn_rate, momentum=momentum)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model
resnet_model = build_resnet_model()

